In [ ]:
import requests
import pandas as pd
from tqdm import tqdm
import time
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data collection

In [ ]:
# Load data
path = '/content/drive/MyDrive/Final Project BBS/data/'
file_name = 'games_data_06122025.csv'
games_df = pd.read_csv(f"{path}{file_name}", encoding = "latin1")
games_df["AppID"] = games_df["AppID"].astype(int)
games_df.shape

(110, 9)

In [ ]:
def get_steam_reviews(app_id, app_name, n_reviews=100):
    """
    Fetch reviews for a given Steam app ID.
    """
    reviews = []
    cursor = '*'
    base_url = f"https://store.steampowered.com/appreviews/{app_id}?json=1&filter=recent&language=english"

    while len(reviews) < n_reviews:
        url = f"{base_url}&cursor={cursor}&num_per_page=100"
        resp = requests.get(url, timeout=10)
        data = resp.json()

        if "reviews" not in data or not data["reviews"]:
            break

        for r in data["reviews"]:
            reviews.append({
                "game": app_name,
                "app_id": app_id,
                "review": r["review"],
                "recommended": r["voted_up"],
                "timestamp_created": r["timestamp_created"],
                "author_playtime": r["author"].get("playtime_forever", None)
            })

        cursor = data.get("cursor", "")
        time.sleep(2)

    return reviews

In [ ]:
all_reviews = []
for _, row in tqdm(games_df.iterrows(), total=len(games_df), desc="Fetching games"):
    game_name = row["Name"]
    app_id = int(row["AppID"])
    reviews = get_steam_reviews(app_id, game_name, n_reviews=100)
    all_reviews.extend(reviews)

df = pd.DataFrame(all_reviews)
df["date"] = pd.to_datetime(df["timestamp_created"], unit="s")
print(f"\nTotal reviews collected: {len(df)}")

Fetching games: 100%|██████████| 110/110 [04:29<00:00,  2.45s/it]


Total reviews collected: 10999


In [ ]:
df.head()

,game,app_id,review,recommended,timestamp_created,author_playtime,date
0,PAYDAY 2,218620,"I rodded bank,then i start game",True,1765018293,1394,2025-12-06 10:51:33
1,PAYDAY 2,218620,money,True,1765016480,12632,2025-12-06 10:21:20
2,PAYDAY 2,218620,Game is a steaming pile that doesn't even work...,False,1765005087,886,2025-12-06 07:11:27
3,PAYDAY 2,218620,clicked no because i was bored,False,1764993460,13063,2025-12-06 03:57:40
4,PAYDAY 2,218620,better than PAYDAY 3,True,1764991399,44,2025-12-06 03:23:19


In [ ]:
# Calculate number of reviews collected by game
df.groupby("app_id").size().value_counts()

,count
100,108
199,1


In [ ]:
# Save raw output
df.to_csv(f"{path}steam_reviews_raw.csv", index=False)

# Data cleaning

In [ ]:
def clean_review(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'<.*?>', '', text)                # remove HTML tags
    text = re.sub(r'http\S+|www\S+', '', text)       # remove URLs
    text = re.sub(r'[^a-zA-ZÀ-ÿ0-9.,!?\'\"\s]', '', text)  # keep letters, numbers & punctuation
    text = re.sub(r'\s+', ' ', text).strip()         # remove extra spaces
    return text

In [ ]:
# Clean reviews
print(f"Total reviews before cleaning: {len(df)}")
df_clean = df.copy()
df_clean["clean_review"] = df_clean["review"].apply(clean_review)
df_clean = df_clean.drop(columns=["review"])
df_clean = df_clean[df_clean["clean_review"].str.strip() != ""]
df_clean = df_clean[df_clean['clean_review'].str.split().str.len() > 1]
print(f"Total reviews after cleaning: {len(df_clean)}")

Total reviews before cleaning: 10999
Total reviews after cleaning: 9182


In [ ]:
# Calculate number of reviews collected by game after cleaning
df_clean.groupby("app_id").size().value_counts().sort_index()

,count
67,1
69,1
70,1
71,2
72,1
74,1
75,3
76,2
77,3
78,7


In [ ]:
# Save cleaned reviews
df_clean.to_csv(f"{path}steam_reviews_clean.csv", index=False)